**Import Required Libraries**

In [0]:
from pyspark.sql import functions as F
from delta.tables import DeltaTable

**Load Project Utilities & Initialize Notebook Widgets**

In [0]:
%run /Workspace/consolidated_pipeline/1_setup/utilities

In [0]:
print(bronze_schema, silver_schema, gold_schema)

bronze silver gold


In [0]:
dbutils.widgets.text("catalog", "fmcg", "Catalog")
dbutils.widgets.text("data_source", "products", "Data Source")

catalog = dbutils.widgets.get("catalog")
data_source = dbutils.widgets.get("data_source")

base_path = f's3://sportsbar-final/{data_source}/*.csv'
print(base_path)

s3://sportsbar-final/products/*.csv


## Bronze

In [0]:
df = (
    spark.read.format("csv")
        .option("header", True)
        .option("inferSchema", True)
        .load(base_path)
        .withColumn("read_timestamp", F.current_timestamp())
        .select("*", "_metadata.file_name", "_metadata.file_size")
)

In [0]:
# print check data type
df.printSchema()

root
 |-- product_name: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- category: string (nullable = true)
 |-- read_timestamp: timestamp (nullable = false)
 |-- file_name: string (nullable = false)
 |-- file_size: long (nullable = false)



In [0]:
display(df.limit(10))

product_name,product_id,category,read_timestamp,file_name,file_size
SportsBar Energy Bar Choco Fudge (60g),25891101,energy bars,2025-11-23T12:20:47.404Z,products.csv,1388
SportsBar Energy Bar Choco Fudge (40g),25891102,energy bars,2025-11-23T12:20:47.404Z,products.csv,1388
SportsBar Energy Bar Choco Fudge (25g),25891103,energy bars,2025-11-23T12:20:47.404Z,products.csv,1388
SportsBar Protien Bar Peanut Crunch (45g),25891201,protien bars,2025-11-23T12:20:47.404Z,products.csv,1388
SportsBar Protien Bar Peanut Crunch (55g),25891202,protien bars,2025-11-23T12:20:47.404Z,products.csv,1388
SportsBar Protien Bar Peanut Crunch (65g),25891203,protien bars,2025-11-23T12:20:47.404Z,products.csv,1388
SportsBar Granola Crunch Honey Almond (400g),25891301,granola & cereals,2025-11-23T12:20:47.404Z,products.csv,1388
SportsBar Granola Crunch Honey Almond (300g),25891302,granola & cereals,2025-11-23T12:20:47.404Z,products.csv,1388
SportsBar Granola Crunch Honey Almond (200g),25891303,granola & cereals,2025-11-23T12:20:47.404Z,products.csv,1388
SportsBar Greek Yogurt Pro Vanilla (200g),25891401,recovery dairy,2025-11-23T12:20:47.404Z,products.csv,1388


In [0]:
df.write\
 .format("delta") \
 .option("delta.enableChangeDataFeed", "true") \
 .mode("overwrite") \
 .saveAsTable(f"{catalog}.{bronze_schema}.{data_source}")

## Silver


In [0]:
df_bronze = spark.sql(f"SELECT * FROM {catalog}.{bronze_schema}.{data_source};")
df_bronze.show(10)

+--------------------+----------+-----------------+--------------------+------------+---------+
|        product_name|product_id|         category|      read_timestamp|   file_name|file_size|
+--------------------+----------+-----------------+--------------------+------------+---------+
|SportsBar Energy ...|  25891101|      energy bars|2025-11-23 12:21:...|products.csv|     1388|
|SportsBar Energy ...|  25891102|      energy bars|2025-11-23 12:21:...|products.csv|     1388|
|SportsBar Energy ...|  25891103|      energy bars|2025-11-23 12:21:...|products.csv|     1388|
|SportsBar Protien...|  25891201|     protien bars|2025-11-23 12:21:...|products.csv|     1388|
|SportsBar Protien...|  25891202|     protien bars|2025-11-23 12:21:...|products.csv|     1388|
|SportsBar Protien...|  25891203|     protien bars|2025-11-23 12:21:...|products.csv|     1388|
|SportsBar Granola...|  25891301|granola & cereals|2025-11-23 12:21:...|products.csv|     1388|
|SportsBar Granola...|  25891302|granola

**Transformations**

- 1: Drop Duplicates

In [0]:
print('Rows before duplicates dropped: ', df_bronze.count())
df_silver = df_bronze.dropDuplicates(['product_id'])
print('Rows after duplicates dropped: ', df_silver.count())

Rows before duplicates dropped:  20
Rows after duplicates dropped:  18


- 2: Title case fix

(energy bars ---> Energy Bars, protien bars ---> Protien Bars etc)

In [0]:
df_silver.select('category').distinct().show()

+-----------------+
|         category|
+-----------------+
|      energy bars|
|     protien bars|
|granola & cereals|
|   recovery dairy|
|   healthy snacks|
|  electrolyte mix|
+-----------------+



In [0]:
# Title case fix
df_silver = df_silver.withColumn(
    "category",
    F.when(F.col("category").isNull(), None)
     .otherwise(F.initcap("category"))
)

In [0]:
df_silver.select('category').distinct().show()

+-----------------+
|         category|
+-----------------+
|      Energy Bars|
|     Protien Bars|
|Granola & Cereals|
|   Recovery Dairy|
|   Healthy Snacks|
|  Electrolyte Mix|
+-----------------+



- 3: Fix Spelling Mistake for `Protien`

In [0]:
# Replace 'protien' → 'protein' in both product_name and category
df_silver = (
    df_silver
    .withColumn(
        "product_name",
        F.regexp_replace(F.col("product_name"), "(?i)Protien", "Protein")
    )
    .withColumn(
        "category",
        F.regexp_replace(F.col("category"), "(?i)Protien", "Protein")
    )
)


In [0]:
display(df_silver.limit(5))

product_name,product_id,category,read_timestamp,file_name,file_size
SportsBar Energy Bar Choco Fudge (60g),25891101,Energy Bars,2025-11-23T12:21:02.933Z,products.csv,1388
SportsBar Energy Bar Choco Fudge (40g),25891102,Energy Bars,2025-11-23T12:21:02.933Z,products.csv,1388
SportsBar Energy Bar Choco Fudge (25g),25891103,Energy Bars,2025-11-23T12:21:02.933Z,products.csv,1388
SportsBar Protein Bar Peanut Crunch (45g),25891201,Protein Bars,2025-11-23T12:21:02.933Z,products.csv,1388
SportsBar Protein Bar Peanut Crunch (55g),25891202,Protein Bars,2025-11-23T12:21:02.933Z,products.csv,1388


### Standardizing Customer Attributes to Match Parent Company Data Model

In [0]:
### 1: Add division column
df_silver = (
    df_silver
    .withColumn(
        "division",
        F.when(F.col("category") == "Energy Bars",        "Nutrition Bars")
         .when(F.col("category") == "Protein Bars",       "Nutrition Bars")
         .when(F.col("category") == "Granola & Cereals",  "Breakfast Foods")
         .when(F.col("category") == "Recovery Dairy",     "Dairy & Recovery")
         .when(F.col("category") == "Healthy Snacks",     "Healthy Snacks")
         .when(F.col("category") == "Electrolyte Mix",    "Hydration & Electrolytes")
         .otherwise("Other")
    )
)


### 2: Variant column
df_silver = df_silver.withColumn(
    "variant",
    F.regexp_extract(F.col("product_name"), r"\((.*?)\)", 1)
)


### 3: Create new column: product_code  

# Invalid product_ids are replaced with a fallback value to avoid losing fact records and ensure downstream joins remain consistent

df_silver = (
    df_silver
    # 1. Generate deterministic product_code from product_name
    .withColumn(
        "product_code",
        F.sha2(F.col("product_name").cast("string"), 256)
    )
    # 2. Clean product_id: keep only numeric IDs, else set to 999999
    .withColumn(
        "product_id",
        F.when(
            F.col("product_id").cast("string").rlike("^[0-9]+$"),
            F.col("product_id").cast("string")
        ).otherwise(F.lit(999999).cast("string"))
    )
    # 3. Rename product_name → product
    .withColumnRenamed("product_name", "product")
)

In [0]:
df_silver = df_silver.select("product_code", "division", "category", "product", "variant", "product_id", "read_timestamp", "file_name", "file_size")

In [0]:
display(df_silver)

product_code,division,category,product,variant,product_id,read_timestamp,file_name,file_size
e91ba9d665f90254da5809bfdebe3db2be01a52f50b6fd96b57eed238392b843,Nutrition Bars,Energy Bars,SportsBar Energy Bar Choco Fudge (60g),60g,25891101,2025-11-23T12:21:02.933Z,products.csv,1388
e92c739a8d78cd6cbe954648c2f9dd75ed61fcfd99b03e10dca65c3082d0728e,Nutrition Bars,Energy Bars,SportsBar Energy Bar Choco Fudge (40g),40g,25891102,2025-11-23T12:21:02.933Z,products.csv,1388
102628255d24304d6bbe0438b1ac992054f262e0814d306d0a34d7356cef3268,Nutrition Bars,Energy Bars,SportsBar Energy Bar Choco Fudge (25g),25g,25891103,2025-11-23T12:21:02.933Z,products.csv,1388
2e387cef1424d6e7b162b45622d4b1a788d11776e33d05cc8552f4ecd2ea1896,Nutrition Bars,Protein Bars,SportsBar Protein Bar Peanut Crunch (45g),45g,25891201,2025-11-23T12:21:02.933Z,products.csv,1388
0cb7b2f42657b625f754e833aa1cf6a967be26f17415f5342302ebb0e90c8a28,Nutrition Bars,Protein Bars,SportsBar Protein Bar Peanut Crunch (55g),55g,25891202,2025-11-23T12:21:02.933Z,products.csv,1388
889c67757ece9c973791dfbc2d47b026a3342cc7255e47a3170329d158e897c2,Nutrition Bars,Protein Bars,SportsBar Protein Bar Peanut Crunch (65g),65g,25891203,2025-11-23T12:21:02.933Z,products.csv,1388
3cab59f05924285270313afcfe40a08983bb03dd88f432e34fc6336914c14345,Breakfast Foods,Granola & Cereals,SportsBar Granola Crunch Honey Almond (400g),400g,25891301,2025-11-23T12:21:02.933Z,products.csv,1388
d9ebd1ca64d23951a6310af93b1c5ac27d831ac842e89aea59a9e8b38621faa5,Breakfast Foods,Granola & Cereals,SportsBar Granola Crunch Honey Almond (300g),300g,25891302,2025-11-23T12:21:02.933Z,products.csv,1388
c68834ceaff15846bc1892c2185dc4e4f471d64fe3796b1a8ecc39a5a48c614f,Breakfast Foods,Granola & Cereals,SportsBar Granola Crunch Honey Almond (200g),200g,25891303,2025-11-23T12:21:02.933Z,products.csv,1388
da6bfc596c1360ca07bda4e0ae6bfe3b8456517fc6e8ddc265630ff940f9ab05,Dairy & Recovery,Recovery Dairy,SportsBar Greek Yogurt Pro Vanilla (200g),200g,25891401,2025-11-23T12:21:02.933Z,products.csv,1388


In [0]:
df_silver.write\
 .format("delta") \
 .option("delta.enableChangeDataFeed", "true") \
 .option("mergeSchema", "true") \
 .mode("overwrite") \
 .saveAsTable(f"{catalog}.{silver_schema}.{data_source}")

## Gold

In [0]:
df_silver = spark.sql(f"SELECT * FROM {catalog}.{silver_schema}.{data_source};")
df_gold = df_silver.select("product_code", "product_id", "division", "category", "product", "variant")
df_gold.show(5)

+--------------------+----------+----------------+--------------+--------------------+-------+
|        product_code|product_id|        division|      category|             product|variant|
+--------------------+----------+----------------+--------------+--------------------+-------+
|2e387cef1424d6e7b...|  25891201|  Nutrition Bars|  Protein Bars|SportsBar Protein...|    45g|
|fe5a8036be4b9a787...|  25891402|Dairy & Recovery|Recovery Dairy|SportsBar Greek Y...|   120g|
|da6bfc596c1360ca0...|  25891401|Dairy & Recovery|Recovery Dairy|SportsBar Greek Y...|   200g|
|e91ba9d665f90254d...|  25891101|  Nutrition Bars|   Energy Bars|SportsBar Energy ...|    60g|
|0cb7b2f42657b625f...|  25891202|  Nutrition Bars|  Protein Bars|SportsBar Protein...|    55g|
+--------------------+----------+----------------+--------------+--------------------+-------+
only showing top 5 rows


In [0]:
df_gold.write\
 .format("delta") \
 .option("delta.enableChangeDataFeed", "true") \
 .mode("overwrite") \
 .saveAsTable(f"{catalog}.{gold_schema}.sb_dim_{data_source}")

## Merging Data source with parent

In [0]:
delta_table = DeltaTable.forName(spark, "fmcg.gold.dim_products")
df_child_products = spark.sql(f"SELECT product_code, division, category, product, variant FROM fmcg.gold.sb_dim_products;")
df_child_products.show(5)

+--------------------+----------------+--------------+--------------------+-------+
|        product_code|        division|      category|             product|variant|
+--------------------+----------------+--------------+--------------------+-------+
|2e387cef1424d6e7b...|  Nutrition Bars|  Protein Bars|SportsBar Protein...|    45g|
|fe5a8036be4b9a787...|Dairy & Recovery|Recovery Dairy|SportsBar Greek Y...|   120g|
|da6bfc596c1360ca0...|Dairy & Recovery|Recovery Dairy|SportsBar Greek Y...|   200g|
|e91ba9d665f90254d...|  Nutrition Bars|   Energy Bars|SportsBar Energy ...|    60g|
|0cb7b2f42657b625f...|  Nutrition Bars|  Protein Bars|SportsBar Protein...|    55g|
+--------------------+----------------+--------------+--------------------+-------+
only showing top 5 rows


In [0]:
delta_table.alias("target").merge(
    source=df_child_products.alias("source"),
    condition="target.product_code = source.product_code"
).whenMatchedUpdate(
    set={
        "division": "source.division",
        "category": "source.category",
        "product": "source.product",
        "variant": "source.variant"
    }
).whenNotMatchedInsert(
    values={
        "product_code": "source.product_code",
        "division": "source.division",
        "category": "source.category",
        "product": "source.product",
        "variant": "source.variant"
    }
).execute()

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]